<a href="https://colab.research.google.com/github/MarielaNina/-Guide-to-Advanced-LLM-Techniques-Public/blob/main/M%C3%B3dulo_1_LangChain_e_Sa%C3%ADdas_Estruturadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Módulo 1: A Base - LangChain e Saídas Estruturadas

# Introdução

Bem-vindo ao primeiro módulo do nosso tutorial! O objetivo aqui é construir a fundação para todas as técnicas avançadas que exploraremos. Antes de mergulhar em engenharia de prompt, ensembles ou fine-tuning, precisamos garantir que conseguimos nos comunicar com os Modelos de Linguagem de Grande Porte (LLMs) de forma eficiente, robusta e, mais importante, programática.

Neste notebook, vamos abordar três pilares essenciais:
1. **Configuração do Ambiente:** Como acessar LLMs poderosos através de APIs, com foco em opções gratuitas.
2. **Orquestração com LangChain:** Uma introdução à biblioteca LangChain, que simplifica a criação de aplicações com LLMs.
3. **Geração de Saídas Estruturadas:** A técnica crucial para transformar as respostas em texto livre dos LLMs em formatos de dados consistentes e utilizáveis, como JSON, que são a base para qualquer aplicação real.

Ao final deste módulo, você terá um ambiente configurado e será capaz de instruir um LLM a retornar informações em um formato Python pré-definido, pronto para ser integrado em qualquer software.

# 1. Configuração das APIs

Para interagir com a maioria dos LLMs de ponta, utilizamos uma Interface de Programação de Aplicações (API). Ela funciona como uma "ponte" que permite que nosso código envie requisições para o modelo e receba as respostas.

## 1.1. Groq

Groq é uma plataforma de inferência de IA de alta performance. Ela oferece acesso via API a diversos modelos open-source de ponta.

Ela possui 2 atrativos principais. O primeiro é que ela oferece um plano gratuito que permite rodar LLMs poderosos de forma gratuita (dentro de certos limites de uso). O segundo é a velocidade de processamento, que é extremamente rápida.

Você pode criar sua conta gratuitamente ([link](console.groq.com)) e, após o login, gerar sua chave de API na aba “API Keys”. Depois de se cadastrar, siga o seguinte passo a passo:
1. Acesse a aba **Secrets** (representada por um ícone de chave na barra lateral do Colab)
2. Clique em **"Adicionar novo secret"**
3. Defina o Nome `"GROQ_API_KEY"`
4. Cole a chave de API que você gerou na plataforma

Neste tutorial, vamos utilizar a API da Groq, mas voce pode usar qualquer outro modelo com integração com o langchain.

In [14]:
!pip install -q langchain langchain-core langchain-community langchain-groq

In [16]:
from langchain_groq import ChatGroq
from google.colab import userdata

llm_groq = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.7
)

## 1.2. Sabiá / Maritaca.AI

É uma plataforma brasileira que disponibiliza modelos de linguagem treinados em português com uma API compatível com a da OpenAI. Ao se cadastrar no serviço [(link)](https://www.maritaca.ai/), você ganha R$20,00 de crédito grátis após verificar um método de pagamento.

Esse crédito inicial é suficiente para realizar o tutorial. Opcionalmente, você pode fazer uma pequena recarga (mesmo o valor mínimo) porque isso aumenta a velocidade de geração e eleva o limite de requisições por minuto consideravelmente. Não é obrigatório para seguir o tutorial, mas se você notar lentidão, essa pode ser uma solução.

Por ser compatível com a API da OpenAI, usar o Sabiá é muito fácil: você pode utilizar bibliotecas pensadas para OpenAI apenas substituindo a chave de API e endpoint. Além disso, o Sabiá já suporta funcionalidades avançadas como saídas estruturadas e chamada de função de forma nativa. Ou seja, é uma alternativa local/nacional para usar LLMs poderosos a custo menor e com suporte ao português.

Depois de se cadastrar, adicione a chave `MARITALK_API_KEY` á aba Secrets do Colab seguindo os mesmos passos do Groq.

In [42]:
from langchain_community.chat_models import ChatMaritalk

llm_sabia = ChatMaritalk(
    model="sabiazinho-3",
    api_key=userdata.get('MARITALK_API_KEY'),
    temperature=0.7,
)

# 2. Introdução ao LangChain

## 2.1. Interagindo com o Modelo

Construir aplicações com LLMs envolve mais do que apenas enviar um prompt para uma API. Frequentemente, precisamos encadear múltiplas chamadas, gerenciar o histórico da conversa, conectar o LLM a fontes de dados externas e formatar suas saídas. Fazer tudo isso manualmente pode ser complexo e repetitivo.

É aqui que entra o LangChain [1]. LangChain é um framework de código aberto projetado para simplificar o desenvolvimento de aplicações baseadas em LLMs. Ele fornece um conjunto de abstrações e componentes modulares que facilitam a criação de cadeias (chains) e agentes complexos.

Já vimos como instanciar ambos os modelos, para chamar eles no nosso código bastam chamar o método invoke com uma string.

In [46]:
resposta = llm_groq.invoke("Olá, tudo bem?")
print(resposta)

content='<think>\n\n</think>\n\nOlá! Sim, estou bem, obrigado. Como posso ajudar você hoje? 😊' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 9, 'total_tokens': 36, 'completion_time': 0.096885119, 'prompt_time': 0.010130248, 'queue_time': 2.477559818, 'total_time': 0.107015367}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_e98d30d035', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--51beb9ed-7ab7-44e4-8fa2-2e6ffb8765cf-0' usage_metadata={'input_tokens': 9, 'output_tokens': 27, 'total_tokens': 36}


Com apenas uma linha de código, podemos interagir com as llms, com toda a complexidade da comunicação via API abstraída pelo LangChain.

## 2.2. Generalização de Prompts
O langchain também oferece uma forma de criar prompts dinâmicos reutilizaveis, fazemos isso através do prompt template.

In [48]:
from langchain_core.prompts import PromptTemplate

template = "Escreva um poema {tamanho} sobre {tema}."
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['tamanho', 'tema'], input_types={}, partial_variables={}, template='Escreva um poema {tamanho} sobre {tema}.')

A maioria das interações através do langchain acontece através do método `invoke`. Podemos gerar o prompt para a LLM da seguinte forma:

In [49]:
poema = prompt.invoke({"tamanho": "curto", "tema": "a lua"})
poema

StringPromptValue(text='Escreva um poema curto sobre a lua.')

## 2.3. Cadeias (Chains)

Agora que temos um prompt dinâmico e um llm, podemos combiná-los. A forma mais idiomática e poderosa de fazer isso no LangChain é através de `chains`, utilizando o operador pipe (`|`). Isso cria um fluxo de dados legível e modular, onde a saída de um componente se torna automaticamente a entrada do próximo, simplificando a lógica.

In [20]:
from langchain_core.output_parsers import StrOutputParser

In [21]:
entrada = prompt.invoke({"tamanho": "curto", "tema": "a lua"})
resposta = llm_groq.invoke(entrada)

parser = StrOutputParser()
resposta = parser.invoke(resposta)

print(resposta)

<think>
Okay, I need to write a short poem about the moon. Let me think about what the moon represents. It's often associated with night, beauty, mystery, and sometimes loneliness. I should use some imagery that evokes these feelings.

Maybe start with the moon in the sky, its glow. I can describe its color, like silver or white. Then, perhaps talk about its reflection on water, which is a common poetic image.

I should also consider the phases of the moon, maybe mention crescent or full moon. This adds variety to the poem. I can talk about how the moon affects the night, maybe personify it a bit, giving it actions or emotions.

Rhyme scheme is important. Let's go with something simple, like ABAB or AABB. Each stanza can have four lines. I'll keep the language simple and flowing, avoiding complicated words so it's easy to understand and has a nice rhythm.

Let me brainstorm some lines. First stanza: Introduce the moon in the sky, its glow, maybe its silence. Second stanza: Its reflecti

In [22]:
chain = prompt | llm_groq | StrOutputParser()
resposta = chain.invoke({"tamanho": "curto", "tema": "a lua"})
print(resposta)

<think>
Okay, I need to write a short poem about the moon. Hmm, where do I start? Well, the moon is such a beautiful and timeless subject. I should think about its characteristics. It's often associated with night, light, and maybe a bit of mystery.

Let me brainstorm some words related to the moon: glowing, silver, crescent, full moon, phases, night, sky, stars, light, shadow, dream, silent, beacon. That should give me a good foundation.

Now, I want the poem to flow well, so maybe I can structure it in a few stanzas. Let's see, perhaps start with describing the moon in the sky, then move on to its phases and its effect on the night.

I should also think about the rhythm and rhyme. Maybe a simple AABB rhyme scheme would work well for a short poem. Each stanza can have four lines, with the second and fourth lines rhyming.

First stanza: Introduce the moon in the sky. Something like "The moon ascends the velvet sky," and then another line about its glow. Maybe "With gentle glow, it catc

# 3. Saídas Estruturadas

## 3.1. A Necessidade de Saídas Estruturadas

Um dos maiores desafios ao usar LLMs em aplicações de software é que, por natureza, eles geram texto não estruturado. Para uma tarefa de classificação de sentimento, um LLM pode responder:
1. "O sentimento é positivo"
2. "Positivo"
3. "Com base na análise, o texto expressa um sentimento positivo."
4. ...

Todas essas respostas são corretas para um humano, mas a variação torna difícil para um programa processá-las de forma confiável.

Quando pedimos algo a um modelo de linguagem, por padrão ele retorna texto livre, em linguagem natural, o que muitas vezes é suficiente para uma conversa ou resposta direta. No entanto, em aplicações práticas, frequentemente queremos que o modelo nos dê a resposta em um formato específico e estruturado para podermos processá-la automaticamente. Exemplos comuns:
- Preencher campos de um formulário ou banco de dados (por exemplo, extrair de um texto o {"nome": ..., "email": ..., "telefone": ...} em formato JSON).
- Listar informações em forma de tabela (CSV) ou em bullet points bem definidos.
- Retornar um conjunto de pares chave-valor, XML ou outro formato que alguma outra parte do sistema espera.

Ter um formato de saída estruturado torna a integração entre LLMs e sistemas tradicionais muito mais confiável. Se um modelo responde com um parágrafo de texto explicativo, é difícil para um programa extrair exatamente as partes relevantes sem risco de erro. Por outro lado, se conseguimos fazer o modelo responder, por exemplo, em JSON com campos definidos, podemos diretamente carregar esse JSON em uma estrutura de dados na nossa aplicação (um dicionário Python, por exemplo) e utilizar as informações de forma determinística.

$$\text{estrutura} = \text{automação simplificada}.$$

## 3.2. Instrução via Prompt

A abordagem mais direta para obter uma saída estruturada é simplesmente pedir explicitamente no prompt que o modelo formate a resposta de determinada forma. Por exemplo, podemos acrescentar às instruções algo como: "Responda somente no formato JSON, contendo as seguintes chaves...". Essa técnica de prompt engineering muitas vezes resolve casos simples.

In [33]:
template = """Extraia as seguintes informações do currículo abaixo:
- Nome
- Formação
- Anos de experiência

Formate a saída em JSON:
{{
    "nome": "Nome do candidato",
    "formacao": "Formação do candidato",
    "anos_experiencia": "Anos de experiência do candidato"
}}

Currículo:"{resume_text}"

JSON:
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["resume_text"]
)

In [34]:
chain = prompt | llm_groq | StrOutputParser()
resume = "Meu nome é Mariela Nina, tenho 24 anos. Sou Bacharel em Ciência da Computação pela Universidade Federal de Minas Gerais (UFMG). Fui Desenvolvedor de Software na Empresa X por 5 anos e Gestor de Projetos na Empresa Y por 3 anos onde trabalho atualmente."

resposta = chain.invoke({"resume_text": resume})
print(resposta)

<think>
Ok, estou vendo o pedido do usuário. Ele quer que eu extraia informações específicas de um currículo fornecido. As informações necessárias são nome, formação e anos de experiência, e formatar em JSON.

Primeiro, vou ler o currículo cuidadosamente. O nome está logo no início: "Mariela Nina". Ótimo, isso é direto.

Em seguida, a formação. Vejo que ela é Bacharel em Ciência da Computação e menciona a Universidade Federal de Minas Gerais, UFMG. Vou anotar isso.

Agora, anos de experiência. Ela trabalhou como Desenvolvedor de Software na Empresa X por 5 anos e como Gestor de Projetos na Empresa Y por 3 anos. No total, são 5 + 3 = 8 anos. Vou somar esses períodos.

Preciso garantir que o JSON esteja formatado corretamente. Vou organizar as chaves: nome, formacao e anos_experiencia. Certificar que os valores estejam entre aspas e que o JSON inteiro esteja bem estruturado.

Vou revisar para garantir que não haja erros de digitação ou de cálculo. Nome: Mariela Nina. Formação: Bacharel e

Observe que ainda recebemos informações fora da estrutura (pensamento no caso do DeepSeek), porém, o tratamento desse texto é bem mais simples do que a saída comum.

## 3.3. Conversão para Objetos

LangChain oferece uma maneira elegante de implementar saídas estruturadas usando a biblioteca Pydantic. Pydantic permite definir a estrutura dos seus dados usando classes Python. LangChain, então, usa essa definição para:
1. Gerar automaticamente uma instrução de formatação para o LLM.
2. Analisar a saída de texto do LLM e convertê-la em um objeto Python.

O Pydantic é uma biblioteca Python para validação de dados e gerenciamento de configurações. Ela permite definir a estrutura de dados desejada usando classes Python normais, com tipos de dados forçados. Para nosso propósito, sua principal vantagem é a capacidade de criar 'modelos de dados' que o LangChain pode usar para instruir o LLM sobre como formatar sua saída e, em seguida, validar se a saída do modelo corresponde a essa estrutura.

In [35]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class InfoCurriculo(BaseModel):
    nome: str = Field(description="Nome do candidato")
    formacao: str = Field(description="Formação do candidato")
    anos_experiencia: float = Field(description="Anos de experiência do candidato")

parser = PydanticOutputParser(pydantic_object=InfoCurriculo)
formato = parser.get_format_instructions()
print(formato)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"nome": {"description": "Nome do candidato", "title": "Nome", "type": "string"}, "formacao": {"description": "Formação do candidato", "title": "Formacao", "type": "string"}, "anos_experiencia": {"description": "Anos de experiência do candidato", "title": "Anos Experiencia", "type": "number"}}, "required": ["nome", "formacao", "anos_experiencia"]}
```


In [36]:
template = """Extraia as seguintes informações do currículo abaixo:
- Nome
- Formação
- Anos de experiência

{format_instructions}

Currículo: "{resume_text}"

JSON:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["resume_text"],
    partial_variables={"format_instructions": formato}
)

In [37]:
chain = prompt | llm_groq | parser
resposta = chain.invoke({"resume_text": resume})
resposta


InfoCurriculo(nome='Mariela Nina', formacao='Bacharel em Ciência da Computação pela Universidade Federal de Minas Gerais (UFMG)', anos_experiencia=8.0)

In [38]:
resposta.nome

'Mariela Nina'

## 3.4. Abordagem Multi-LLM

Embora forçar a formatação em um único passo seja conveniente, a complexidade de realizar duas tarefas simultaneamente — extrair a informação e formatá-la perfeitamente — pode, por vezes, degradar a qualidade do resultado. Para mitigar isso, podemos decompor o problema: uma primeira chamada ao LLM foca apenas em extrair a informação em linguagem natural, e uma segunda chamada, que pode usar um modelo mais simples e rápido, foca exclusivamente em converter essa extração para o formato JSON desejado.

In [39]:
template_extracao = """Extraia as seguintes informações do currículo abaixo:
- Nome
- Formação
- Anos de experiência

Apresente as informações extraídas de forma clara.

Currículo:
{curriculo}
"""
prompt_extracao = PromptTemplate.from_template(template_extracao)

chain_extracao = prompt_extracao | llm_groq | StrOutputParser()

In [40]:
template_formatacao = """
Formate a informação extraída abaixo para um objeto JSON.
Siga estritamente o esquema JSON fornecido.

Informação Extraída:
{info_extraida}

Esquema JSON:
{esquema_json}
"""
prompt_formatacao = PromptTemplate.from_template(template_formatacao)

# Modelo LLM para a segunda etapa (pode ser um modelo mais rápido/barato).
llm_formatador = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.0 # Temperatura 0 para a tarefa de formatação, que deve ser determinística.
)


In [41]:
chain_multi_llm = (
    {
        "info_extraida": chain_extracao,
        "esquema_json": lambda x: formato
    }
    | prompt_formatacao
    | llm_formatador
    | parser
)

resposta = chain_multi_llm.invoke(resume)
resposta

InfoCurriculo(nome='Mariela Nina', formacao='Bacharel em Ciência da Computação pela Universidade Federal de Minas Gerais (UFMG)', anos_experiencia=8.0)

# 4. Atividade Prática: Classificador de Notícias Financeiras

Agora é sua vez de aplicar o que aprendeu! Vamos usar um pequeno conjunto de dados de notícias financeiras para treinar suas habilidades.

**Seu objetivo:** Criar uma chain que recebe uma frase de uma notícia e retorna um objeto estruturado com sua polaridade e emocao.

Passos:
1. **Defina a Estrutura:** Crie uma classe Pydantic chamada ClassificacaoNoticia com os campos nescessários.
    - O dataset contém notícias financeiras em português.
    - Cada notícia foi resumida em 3 frases, que representam começo (f1), meio (f2) e fim (f3).
    - Cada frase possui uma polaridade (positivo, neutro, negativo) e uma emoção universal do ekman (felicidade, tristeza, raiva, nojo, medo, surpreza e desprezo)
2. **Crie o Parser:** Instancie um PydanticOutputParser com base na sua classe.
3. **Crie o Prompt:** Elabore um PromptTemplate que instrua o LLM a classificar o sentimento de um texto de notícia, usando as instruções de formato do seu parser.
4. **Construa a Chain:** Utilize a abordagem Multi-LLM com conversão para objetos com uma única chamada.
5. **Teste:** Execute sua implementação para alguns elementos do dataset e imprima os resultados.

In [32]:
import pandas as pd

file_path = "/content/drive/MyDrive/Curso LLMs/dataset.csv"
df = pd.read_csv(file_path)
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Curso LLMs/dataset.csv'

In [ ]:
class ClassificacaoNoticia(BaseModel):
    pass

# Conclusão do Módulo

Parabéns! Você configurou seu ambiente, aprendeu a usar o LangChain para interagir com um LLM e, o mais importante, implementou uma forma robusta de obter saídas estruturadas. O objeto `ClassificacaoSentimento` que recebemos no final é previsível e fácil de usar em qualquer sistema.

Essa habilidade é o alicerce sobre o qual construiremos técnicas mais sofisticadas. No próximo módulo, vamos nos aprofundar na Engenharia de Prompt para melhorar drasticamente a qualidade e a precisão das respostas do modelo.

# Referências

[1] Chase, H. (2022). LangChain. GitHub. https://github.com/langchain-ai/langchain